In [308]:
import pandas as pd
pd.set_option('display.max_columns', 50000)
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import datetime, os

In [309]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Num GPUs Available:  1


In [310]:
df = pd.read_csv("./dataset/train.csv")
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [311]:
df = df[df.churn_risk_score != -1]
df.describe()

,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,churn_risk_score
count,35829.000000,35829.000000,35829.000000,35829.000000,32488.000000,35829.000000
mean,37.120266,-42.143906,243.373859,29304.272306,686.849231,3.608278
std,15.865360,229.254856,397.784650,19484.565419,194.574486,1.176426
min,10.000000,-999.000000,-2814.109110,800.460000,-760.661236,1.000000
25%,23.000000,8.000000,59.810000,14194.650000,615.967500,3.000000
50%,37.000000,12.000000,161.620000,27584.530000,697.820000,4.000000
75%,51.000000,16.000000,356.330000,40874.010000,764.002500,5.000000
max,64.000000,26.000000,3235.578521,99914.050000,2069.069761,5.000000


In [312]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35829 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   35829 non-null  object 
 1   Name                          35829 non-null  object 
 2   age                           35829 non-null  int64  
 3   gender                        35829 non-null  object 
 4   security_no                   35829 non-null  object 
 5   region_category               30566 non-null  object 
 6   membership_category           35829 non-null  object 
 7   joining_date                  35829 non-null  object 
 8   joined_through_referral       35829 non-null  object 
 9   referral_id                   35829 non-null  object 
 10  preferred_offer_types         35553 non-null  object 
 11  medium_of_operation           35829 non-null  object 
 12  internet_option               35829 non-null  object 
 13  l

In [313]:
def valueCounts(df):
    print(df["gender"].value_counts())
    print("-"*50)
    print(df["region_category"].value_counts())
    print("-"*50)
    print(df["membership_category"].value_counts())
    print("-"*50)
    print(df["joined_through_referral"].value_counts())
    print("-"*50)
    print(df["preferred_offer_types"].value_counts())
    print("-"*50)
    print(df["medium_of_operation"].value_counts())
    print("-"*50)
    print(df["internet_option"].value_counts())
    print("-"*50)
    print(df["used_special_discount"].value_counts())
    print("-"*50)
    print(df["offer_application_preference"].value_counts())
    print("-"*50)
    print(df["past_complaint"].value_counts())
    print("-"*50)
    print(df["complaint_status"].value_counts())
    print("-"*50)
    print(df["feedback"].value_counts())
    print("-"*50)
    print(df["avg_frequency_login_days"].value_counts())
    print("-"*50)
    print(df["churn_risk_score"].value_counts())
    print("-"*50)
    
valueCounts(df)

F          17892
M          17881
Unknown       56
Name: gender, dtype: int64
--------------------------------------------------
Town       13702
City       12315
Village     4549
Name: region_category, dtype: int64
--------------------------------------------------
Basic Membership       7473
No Membership          7466
Gold Membership        6574
Silver Membership      5806
Premium Membership     4308
Platinum Membership    4202
Name: membership_category, dtype: int64
--------------------------------------------------
No     15354
Yes    15183
?       5292
Name: joined_through_referral, dtype: int64
--------------------------------------------------
Gift Vouchers/Coupons       11977
Credit/Debit Card Offers    11860
Without Offers              11716
Name: preferred_offer_types, dtype: int64
--------------------------------------------------
Desktop       13476
Smartphone    13444
?              5230
Both           3679
Name: medium_of_operation, dtype: int64
-------------------------

In [314]:
df.isnull().values.any()

True

In [315]:
df.isnull().sum()

customer_id                        0
Name                               0
age                                0
gender                             0
security_no                        0
region_category                 5263
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            276
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3341
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [316]:
# setting target wrong value -1 to 1 assuming sign issue, 
# df['churn_risk_score'] = df['churn_risk_score'].apply(lambda x:1 if x == -1 else x)
# df['churn_risk_score'] = df['churn_risk_score'].apply(lambda x:x+1)

In [317]:
def handleMissingValues(df):
    
    df['joining_date']  = pd.to_datetime(df['joining_date'],format="%Y-%m-%d")
    
    
    # replacing Unknown words with most occured in df for gender column
    df['gender'] = df['gender'].replace(['Unknown'], df["gender"].mode()[0])
    df['joined_through_referral'] = df['joined_through_referral'].replace(['?'], df["joined_through_referral"].mode()[0])
    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].replace(['Error'], None)
    
    
    df["region_category"] = df["region_category"].fillna(df["region_category"].mode()[0])
    df["preferred_offer_types"] = df["preferred_offer_types"].fillna(df["preferred_offer_types"].mode()[0])
    df["points_in_wallet"] = df["points_in_wallet"].fillna(df["points_in_wallet"].median())
    df["avg_frequency_login_days"] = df["avg_frequency_login_days"].fillna(df["avg_frequency_login_days"].median())
    
    
    df["avg_frequency_login_days"] = pd.to_numeric(df["avg_frequency_login_days"])
    
    return df

df = handleMissingValues(df)
df.isnull().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

In [318]:
valueCounts(df)

F    17948
M    17881
Name: gender, dtype: int64
--------------------------------------------------
Town       18965
City       12315
Village     4549
Name: region_category, dtype: int64
--------------------------------------------------
Basic Membership       7473
No Membership          7466
Gold Membership        6574
Silver Membership      5806
Premium Membership     4308
Platinum Membership    4202
Name: membership_category, dtype: int64
--------------------------------------------------
No     20646
Yes    15183
Name: joined_through_referral, dtype: int64
--------------------------------------------------
Gift Vouchers/Coupons       12253
Credit/Debit Card Offers    11860
Without Offers              11716
Name: preferred_offer_types, dtype: int64
--------------------------------------------------
Desktop       13476
Smartphone    13444
?              5230
Both           3679
Name: medium_of_operation, dtype: int64
--------------------------------------------------
Wi-Fi          1

In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35829 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   customer_id                   35829 non-null  object        
 1   Name                          35829 non-null  object        
 2   age                           35829 non-null  int64         
 3   gender                        35829 non-null  object        
 4   security_no                   35829 non-null  object        
 5   region_category               35829 non-null  object        
 6   membership_category           35829 non-null  object        
 7   joining_date                  35829 non-null  datetime64[ns]
 8   joined_through_referral       35829 non-null  object        
 9   referral_id                   35829 non-null  object        
 10  preferred_offer_types         35829 non-null  object        
 11  medium_of_operation         

In [320]:
def handleExtraFeatures(df):

    from datetime import date
    df['todays_date'] = date.today().strftime("%Y-%m-%d")
    df['todays_date']  = pd.to_datetime(df['todays_date'],format="%Y-%m-%d")
    

    df['customer_days'] = df['todays_date'] - df['joining_date'] 
    df['customer_days'] = df['customer_days'] / np.timedelta64(1,'D')

    return df
df = handleExtraFeatures(df)
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,No,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.82,Yes,No,Yes,Solved,Quality Customer Care,1,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5,2021-03-27,1292.0


In [321]:
def labelEncoding(df, onlyTransform: False, label_object = {}):
    categorical_columns = [
        'gender',
        'joined_through_referral',
        'used_special_discount',
        'offer_application_preference',
        'past_complaint'
    ]
    for col in categorical_columns:
        if col in df.columns:
            labelencoder = None
            if onlyTransform is False:
                labelencoder = LabelEncoder()
                labelencoder.fit(df[col])
                label_object[col] = labelencoder
            else:
                labelencoder = label_object[col]

            df[col] = labelencoder.transform(df[col])
    return df, label_object

df, label_object = labelEncoding(df, onlyTransform= False, label_object= {})
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,Village,Platinum Membership,2017-08-17,0,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,2,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,City,Premium Membership,2017-08-28,0,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.82,1,0,1,Solved,Quality Customer Care,1,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,Town,No Membership,2016-11-11,1,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,5,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,City,No Membership,2016-10-29,1,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,5,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,City,No Membership,2017-09-12,0,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,5,2021-03-27,1292.0


In [322]:
def encodeMembership(df):
    
    cleanup_nums = {
                    "membership_category":{
                        "No Membership":0,
                        "Basic Membership":1,
                        "Gold Membership":2,
                        "Silver Membership":3,
                        "Premium Membership":4,
                        "Platinum Membership":5
                    }
                   }
    
    df = df.replace(cleanup_nums)
    
    return df

df = encodeMembership(df)
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,Village,5,2017-08-17,0,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,2,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,City,4,2017-08-28,0,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.82,1,0,1,Solved,Quality Customer Care,1,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,Town,0,2016-11-11,1,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,5,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,City,0,2016-10-29,1,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,5,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,City,0,2017-09-12,0,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,5,2021-03-27,1292.0


In [323]:
def oneHotEncoding(df, onlyTransform=False, one_hot_object = {}):
    categorical_columns = [
        'region_category',
        'preferred_offer_types',
        'medium_of_operation',
        'internet_option',
        'complaint_status',
        'feedback'
    ]
    for col in categorical_columns:
        
        col_data  = df[col].values.reshape(-1,1)
        
        if onlyTransform is False:
            onehot_encoder = OneHotEncoder()
            onehot_encoder = onehot_encoder.fit(col_data)
            one_hot_object[col] = onehot_encoder
        
        onehot_encoded = one_hot_object[col].transform(col_data).toarray()
        columns_list = []
        for i in range(onehot_encoded.shape[1]):
            columns_list.append(col+"_"+str(int(i)))
        dfOneHot = pd.DataFrame(onehot_encoded, columns = columns_list) 
        df = pd.concat([df, dfOneHot], axis=1)
        

    df = df.drop(categorical_columns,axis=1)
    return df, one_hot_object


df, one_hot_object = oneHotEncoding(df, onlyTransform=False, one_hot_object = {})
df.head()

,customer_id,Name,age,gender,security_no,membership_category,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,todays_date,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,preferred_offer_types_3,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,medium_of_operation_4,internet_option_0,internet_option_1,internet_option_2,internet_option_3,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,complaint_status_5,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8,feedback_9
0,fffe4300490044003600300030003800,Pattie Morrisey,18.0,0.0,XW0DQ7H,5.0,2017-08-17,0.0,xxxxxxxx,16:08:02,17.0,300.63,53005.25,17.0,781.75,1.0,1.0,0.0,2.0,2021-03-27,1318.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,fffe43004900440032003100300035003700,Traci Peery,32.0,0.0,5K0N3X1,4.0,2017-08-28,0.0,CID21329,12:38:13,16.0,306.34,12838.38,10.0,697.82,1.0,0.0,1.0,1.0,2021-03-27,1307.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44.0,0.0,1F2TCL3,0.0,2016-11-11,1.0,CID12313,22:53:21,14.0,516.16,21027.00,22.0,500.69,0.0,1.0,1.0,5.0,2021-03-27,1597.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37.0,1.0,VJGJ33N,0.0,2016-10-29,1.0,CID3793,15:57:50,11.0,53.27,25239.56,6.0,567.66,0.0,1.0,1.0,5.0,2021-03-27,1610.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31.0,0.0,SVZXCWB,0.0,2017-09-12,0.0,xxxxxxxx,15:46:44,20.0,113.13,24483.66,16.0,663.06,0.0,1.0,1.0,5.0,2021-03-27,1292.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [324]:
def removeRedundantColumns(df):
    
    redundant_columns = ['customer_id','Name','security_no','joining_date','referral_id','last_visit_time','todays_date']

    df = df.drop(redundant_columns,axis=1)
    
    return df

df = removeRedundantColumns(df)
df.head()

,age,gender,membership_category,joined_through_referral,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,preferred_offer_types_3,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,medium_of_operation_4,internet_option_0,internet_option_1,internet_option_2,internet_option_3,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,complaint_status_5,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8,feedback_9
0,18.0,0.0,5.0,0.0,17.0,300.63,53005.25,17.0,781.75,1.0,1.0,0.0,2.0,1318.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,32.0,0.0,4.0,0.0,16.0,306.34,12838.38,10.0,697.82,1.0,0.0,1.0,1.0,1307.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,44.0,0.0,0.0,1.0,14.0,516.16,21027.00,22.0,500.69,0.0,1.0,1.0,5.0,1597.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,37.0,1.0,0.0,1.0,11.0,53.27,25239.56,6.0,567.66,0.0,1.0,1.0,5.0,1610.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31.0,0.0,0.0,0.0,20.0,113.13,24483.66,16.0,663.06,0.0,1.0,1.0,5.0,1292.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [325]:
y = pd.DataFrame(df["churn_risk_score"])
y_label_encoder = LabelEncoder()
y_label_encoder = y_label_encoder.fit(y)
y = y_label_encoder.transform(y)
y = tf.keras.utils.to_categorical(y)
df = df.drop("churn_risk_score", axis=1)

X = pd.DataFrame(df)
print(X.shape)
print(y.shape)

(36992, 45)
(36992, 6)


c:\users\pitbox\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [326]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

X_train = X
y_train = y
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(36992, 45)
(36992, 6)
(11098, 45)
(11098, 6)


In [327]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler

scalerX = StandardScaler()
scalerX = scalerX.fit(X_train)
X_train = scalerX.transform(X_train)
X_train = pd.DataFrame(X_train)

X_test = scalerX.transform(X_test)
X_test = pd.DataFrame(X_test)

In [328]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,-1.205175,-0.998132,1.739518,-0.857553,0.257987,0.143940,1.216415,0.111747,0.506611,0.903920,0.900611,-0.994850,-1.285114,-0.723692,-1.060464,2.622258,-0.687544,1.419685,-0.681405,-0.176804,2.464315,-0.332326,-0.757021,-0.755608,-0.180088,-0.686956,-0.690924,1.441722,-0.180166,-0.367948,1.026797,-0.370595,-0.369462,-0.372197,-0.180166,-0.443145,-0.442447,-0.446633,-0.442621,5.148140,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
1,-0.322737,-0.998132,1.133619,-0.857553,0.253625,0.158294,-0.845085,-0.649987,0.053683,0.903920,-1.110357,1.005177,-1.319720,1.381803,-1.060464,-0.381351,-0.687544,1.419685,-0.681405,-0.176804,-0.405792,-0.332326,1.320967,-0.755608,-0.180088,-0.686956,1.447338,-0.693615,-0.180166,-0.367948,-0.973902,2.698365,-0.369462,-0.372197,-0.180166,-0.443145,-0.442447,-0.446633,-0.442621,-0.194245,5.198485,-0.197001,-0.444193,-0.19432,-0.180166
2,0.433638,-0.998132,-1.289977,1.166109,0.244901,0.685773,-0.424818,0.655843,-1.010129,-1.106293,0.900611,1.005177,-0.407391,-0.723692,0.942983,-0.381351,-0.687544,1.419685,-0.681405,-0.176804,-0.405792,-0.332326,1.320967,-0.755608,-0.180088,-0.686956,-0.690924,1.441722,-0.180166,-0.367948,-0.973902,-0.370595,2.706641,-0.372197,-0.180166,-0.443145,-0.442447,-0.446633,2.259268,-0.194245,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
3,-0.007581,1.001872,-1.289977,1.166109,0.231815,-0.477913,-0.208615,-1.085263,-0.648725,-1.106293,0.900611,1.005177,-0.366493,1.381803,-1.060464,-0.381351,-0.687544,1.419685,-0.681405,-0.176804,-0.405792,-0.332326,1.320967,-0.755608,-0.180088,-0.686956,1.447338,-0.693615,-0.180166,-0.367948,-0.973902,-0.370595,-0.369462,2.686749,-0.180166,-0.443145,-0.442447,-0.446633,2.259268,-0.194245,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
4,-0.385768,-0.998132,-1.289977,-0.857553,0.271073,-0.327428,-0.247410,0.002928,-0.133899,-1.106293,0.900611,1.005177,-1.366910,1.381803,-1.060464,-0.381351,1.454452,-0.704381,-0.681405,-0.176804,-0.405792,-0.332326,-0.757021,1.323438,-0.180088,-0.686956,1.447338,-0.693615,-0.180166,-0.367948,-0.973902,2.698365,-0.369462,-0.372197,-0.180166,-0.443145,-0.442447,-0.446633,2.259268,-0.194245,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,0.559701,-0.998132,-0.684078,-0.857553,0.192557,-2.247621,-0.104012,-1.085263,-0.260987,-1.106293,0.900611,1.005177,-1.395223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.405792,-0.332326,-0.757021,1.323438,-0.180088,-0.686956,-0.690924,1.441722,-0.180166,2.717776,-0.973902,-0.370595,-0.369462,-0.372197,-0.180166,2.256596,-0.442447,-0.446633,-0.442621,-0.194245,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
36988,-0.511831,-0.998132,-0.684078,-0.857553,0.240539,-2.216047,-0.935860,1.308757,-0.862805,0.903920,-1.110357,-0.994850,0.023606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.405792,-0.332326,1.320967,-0.755608,-0.180088,-0.686956,-0.690924,1.441722,-0.180166,-0.367948,1.026797,-0.370595,-0.369462,-0.372197,-0.180166,-0.443145,2.260160,-0.446633,-0.442621,-0.194245,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
36989,-0.890018,-0.998132,-0.684078,1.166109,0.236177,-0.222319,0.452841,1.308757,-0.039946,-1.106293,0.900611,1.005177,-0.215487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.405792,-0.332326,-0.757021,1.323438,-0.180088,-0.686956,-0.690924,1.441722,-0.180166,-0.367948,-0.973902,-0.370595,-0.369462,2.686749,-0.180166,-0.443145,-0.442447,-0.446633,2.259268,-0.194245,-0.192364,-0.197001,-0.444193,-0.19432,-0.180166
36990,1.000920,1.001872,1.739518,-0.857553,0.249263,0.601430,-1.381903,0.438205,-2.647566,0.903920,0.900611,-0.994850,-1.086919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.405792,-0.332326,1.320967,-0.755608,-0.180088,-0.686956,1.447338,-0.693615,-0.180166,-0.367948,1.026797,-0.370595,-0.369462

In [329]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout 
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(256,input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(128, activation='relu',kernel_regularizer=L1L2()))
model.add(Dense(64, activation='relu',kernel_regularizer=L1L2()))
model.add(Dense(64, activation='relu',kernel_regularizer=L1L2()))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy',get_f1])

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 256)               11776     
_________________________________________________________________
dense_63 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_64 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_65 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_66 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_67 (Dense)             (None, 6)                 198       
Total params: 59,366
Trainable params: 59,366
Non-trainable params: 0
__________________________________________________

In [330]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=1000,epochs=50,verbose=1)

Epoch 1/50
37/37 [==============================] - 2s 25ms/step - loss: 1.7746 - accuracy: 0.2574 - get_f1: 0.3925 - val_loss: 1.6300 - val_accuracy: 0.2754 - val_get_f1: 0.4142
Epoch 2/50
37/37 [==============================] - 0s 13ms/step - loss: 1.6033 - accuracy: 0.2773 - get_f1: 0.4109 - val_loss: 1.5621 - val_accuracy: 0.2818 - val_get_f1: 0.4142
Epoch 3/50
37/37 [==============================] - 1s 14ms/step - loss: 1.5619 - accuracy: 0.2820 - get_f1: 0.4104 - val_loss: 1.5558 - val_accuracy: 0.2818 - val_get_f1: 0.4142
Epoch 4/50
37/37 [==============================] - 0s 13ms/step - loss: 1.5567 - accuracy: 0.2843 - get_f1: 0.4107 - val_loss: 1.5551 - val_accuracy: 0.2754 - val_get_f1: 0.4142
Epoch 5/50
37/37 [==============================] - 0s 13ms/step - loss: 1.5569 - accuracy: 0.2827 - get_f1: 0.4108 - val_loss: 1.5548 - val_accuracy: 0.2818 - val_get_f1: 0.4142
Epoch 6/50
37/37 [==============================] - 0s 14ms/step - loss: 1.5563 - accuracy: 0.2760 - get_

37/37 [==============================] - ETA: 0s - loss: 1.5525 - accuracy: 0.2795 - get_f1: 0.412 - 0s 13ms/step - loss: 1.5527 - accuracy: 0.2796 - get_f1: 0.4126 - val_loss: 1.5548 - val_accuracy: 0.2818 - val_get_f1: 0.4142
Epoch 47/50
37/37 [==============================] - 0s 13ms/step - loss: 1.5583 - accuracy: 0.2856 - get_f1: 0.4100 - val_loss: 1.5551 - val_accuracy: 0.2754 - val_get_f1: 0.4142
Epoch 48/50
37/37 [==============================] - 0s 13ms/step - loss: 1.5607 - accuracy: 0.2726 - get_f1: 0.4098 - val_loss: 1.5549 - val_accuracy: 0.2818 - val_get_f1: 0.4142
Epoch 49/50
37/37 [==============================] - 0s 13ms/step - loss: 1.5537 - accuracy: 0.2795 - get_f1: 0.4118 - val_loss: 1.5549 - val_accuracy: 0.2818 - val_get_f1: 0.4142
Epoch 50/50
37/37 [==============================] - 0s 13ms/step - loss: 1.5568 - accuracy: 0.2831 - get_f1: 0.4104 - val_loss: 1.5548 - val_accuracy: 0.2818 - val_get_f1: 0.4142


In [331]:
prediction = model.predict(X_test)

In [332]:
prediction

array([[0.44163316, 0.44940785, 0.7574416 , 0.7518844 , 0.74903154,
        0.2550631 ],
       [0.44163316, 0.44940785, 0.7574416 , 0.7518844 , 0.74903154,
        0.2550631 ],
       [0.44163316, 0.44940785, 0.7574416 , 0.7518844 , 0.74903154,
        0.2550631 ],
       ...,
       [0.44163316, 0.44940785, 0.7574416 , 0.7518844 , 0.74903154,
        0.2550631 ],
       [0.44163316, 0.44940785, 0.7574416 , 0.7518844 , 0.74903154,
        0.2550631 ],
       [0.44163316, 0.44940785, 0.7574416 , 0.7518844 , 0.74903154,
        0.2550631 ]], dtype=float32)

In [339]:
a = tf.keras.utils.to_categorical([1, 2, 3, 4],num_classes=4)
# a = tf.constant(a, shape=[4, 4])
print(a)

IndexError: index 4 is out of bounds for axis 1 with size 4

In [333]:
y_label = tf.math.argmax(y_test, axis=1)
predict_label = tf.math.argmax(prediction,axis=1)

y_label = y_label_encoder.inverse_transform(y_label)
predict_label = y_label_encoder.inverse_transform(predict_label)

print(y_label)
print(predict_label)

[3. 4. 3. ... 3. 5. 5.]
[3. 3. 3. ... 3. 3. 3.]


In [334]:
from sklearn.metrics import accuracy_score as sk_accuracy_score
from sklearn.metrics import f1_score as sk_f1_score
from sklearn.metrics import precision_score as sk_precision_score
from sklearn.metrics import recall_score as sk_recall_score
from sklearn.metrics import confusion_matrix as sk_confusion_matrix

print("sk_accuracy_score :",100*sk_accuracy_score(y_label, predict_label))
print("sk_f1_score :",100*sk_f1_score(y_label, predict_label,average="macro"))
print("sk_precision_score :", 100*sk_precision_score(y_label, predict_label,average="macro"))
print("sk_recall_score :" ,100*sk_recall_score(y_label, predict_label,average="macro"))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
df_test = pd.read_csv("./dataset/test.csv")
df_test.head()

In [ ]:
df_test =  handleMissingValues(df_test)

In [ ]:
df_test = handleExtraFeatures(df_test)

In [ ]:
df_test, label_object = labelEncoding(df_test, onlyTransform= True, label_object= label_object)

In [ ]:
df_test = encodeMembership(df_test)

In [ ]:
df_test, one_hot_object = oneHotEncoding(df_test, onlyTransform=True, one_hot_object = one_hot_object)

In [ ]:
df_test.head()

In [ ]:
customer_ids = df_test["customer_id"]
customer_ids_df = pd.DataFrame(data = customer_ids,columns=["customer_id"])
df_test = removeRedundantColumns(df_test)
df_test.head()

In [ ]:
x_test_submit = scalerX.transform(df_test)
x_test_submit = pd.DataFrame(x_test_submit)
x_test_submit.head()

In [ ]:
prediction_submit = model.predict(x_test_submit)
predict_label = tf.math.argmax(prediction_submit,axis=1)

predict_label = y_label_encoder.inverse_transform(predict_label)
print(predict_label)
# print(predict_inversed)

In [ ]:
df_to_submit = pd.concat([customer_ids_df,pd.DataFrame(data = predict_label,columns=["churn_risk_score"])], axis=1)
# df_to_submit['churn_risk_score'] = df_to_submit['churn_risk_score'].apply(lambda x:x-1)

In [ ]:
df_to_submit.head()

In [ ]:
print(df_to_submit["churn_risk_score"].value_counts())
print("-"*50)

In [ ]:
df_to_submit.to_csv(path_or_buf="./df_to_submit"+".csv",float_format='%.0f',index=False)